In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
print(train.head())
train.columns
set(list(train["MSSubClass"][:]))
type(list(train["PoolQC"][:])[0])

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

float

In [3]:
def data_prep_dict(dataFrame):
    columns = dataFrame.columns
    getUniqVals = lambda column : set(list(column))
    dataFrameDict = {}
    for column in columns:
        columnUniqVals = getUniqVals(dataFrame.loc[:, column])
        auxDict = {}
        for (idx, val) in enumerate(columnUniqVals):
            if (type(val) == str or val == float("NaN")):
                auxDict[str(val)] = idx 
        dataFrameDict[column] = auxDict
    
    for column in columns:
        if dataFrameDict[column] == {}:
            dataFrameDict.pop(column)
        
        
    return dataFrameDict

print(data_prep_dict(train.head(15)))
trainDict = data_prep_dict(train.head(15))

def data_prep(dataFrame, dataFrameDict):
    dataFrameDictCollapsed = {}
    for key in list(dataFrameDict.keys()):
        dataFrameDictCollapsed.update(dataFrameDict[key])
    dataFrame = dataFrame.replace(dataFrameDictCollapsed)
    return dataFrame

print(data_prep(train.head(15), trainDict))

{'MSZoning': {'RL': 0, 'RM': 1}, 'Street': {'Pave': 0}, 'LotShape': {'IR2': 0, 'Reg': 1, 'IR1': 2}, 'LandContour': {'Lvl': 0}, 'Utilities': {'AllPub': 0}, 'LotConfig': {'Inside': 0, 'Corner': 1, 'FR2': 2}, 'LandSlope': {'Gtl': 0}, 'Neighborhood': {'Veenker': 0, 'NoRidge': 1, 'Somerst': 2, 'Mitchel': 3, 'Sawyer': 4, 'NAmes': 5, 'Crawfor': 6, 'NridgHt': 7, 'BrkSide': 8, 'CollgCr': 9, 'NWAmes': 10, 'OldTown': 11}, 'Condition1': {'Norm': 0, 'Feedr': 1, 'Artery': 2, 'PosN': 3}, 'Condition2': {'Norm': 0, 'Artery': 1}, 'BldgType': {'1Fam': 0, '2fmCon': 1}, 'HouseStyle': {'1.5Unf': 0, '1.5Fin': 1, '1Story': 2, '2Story': 3}, 'RoofStyle': {'Gable': 0, 'Hip': 1}, 'RoofMatl': {'CompShg': 0}, 'Exterior1st': {'MetalSd': 0, 'Wd Sdng': 1, 'VinylSd': 2, 'HdBoard': 3, 'WdShing': 4, 'BrkFace': 5}, 'Exterior2nd': {'MetalSd': 0, 'VinylSd': 1, 'HdBoard': 2, 'Wd Shng': 3, 'Plywood': 4}, 'MasVnrType': {'Stone': 0, 'None': 1, 'BrkFace': 2}, 'ExterQual': {'Gd': 0, 'Ex': 1, 'TA': 2}, 'ExterCond': {'TA': 0}, 'Fou